# Flow ML Course

# TP 5 : HyperParameter Tuning avec Vertex AI TensorBoard

L'objectif de ce TP est de réaliser un hyper-paramétrage de modèe utilisant Vertex AI TensorBoard pour logger les hyper-paramètres et leurs métriques de performances.


### Dataset

Ce tutoriel utilise le dataset [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist).


## Initialisation

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[tensorboard] \
                                 tensorflow==2.15.1

In [ ]:
PROJECT_ID = "projet-ia-448520"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

your_name = ...

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Vertex AI TensorBoard?

Vertex AI TensorBoard est un version de Tensorboard : [Open source TensorBoard](https://www.tensorflow.org/tensorboard/get_started)
(TB). 
C'est un projet Google open source pour la visualisation d'experiment de machine learning : 



*   suivi et visualisation des métriques
*   architecture du modèles (computational graphs i.e ops and layers),
*   projection d'embeddings
*   image, text, audio ...

Vertex AI TensorBoard fournit en plus des fonctionnalités de TensorBoard : 

*  un lien partageable et persistent au dashboard,
*  liste des experiments du projet
*  intégrtion avec les service Vertex AI
*  enterprise-grade security, privacy, and compliance.

Approfondir ici : [Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction).

## Composants TensorBoard et TensorFlow


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/

# Import TensorFlow and the TensorBoard HParams plugin
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

## Download dataset

Télécharge le dataset et noramliser le [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist).

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()



## Your turn 

Normalize X train / test between [0, 1]

In [ ]:
x_train, x_test = ...

## Set up experiment

Réaliser une première experiement en spécifiants les hyper-paramètres suivants :s:

* number of units in the first dense layer
* dropout rate
* optimizer



### Your Turn

Définisser le range des valeurs d'hyper-paramètres
- Utiliser hp.Discrete or hp.RealInterval
- Choisissez votre métriques

Expliquer vos choix : 

...


In [ ]:
HP_NUM_UNITS = hp.HParam("num_units", ...)
HP_DROPOUT = hp.HParam("dropout", ...)
HP_OPTIMIZER = hp.HParam("optimizer", ...)

METRIC_NAME = ...

with tf.summary.create_file_writer("logs/hparam_tuning").as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_NAME, display_name=METRIC_NAME)],
    )

## Adapter les runs Tensorflow 

Définissez l'architecture de votre modèle avec keras. Faites en sorte que les hyper-paramètres soient utilisés dans la fonction d'entrainement.

### Your Turn


In [ ]:
def train_test_model(hparams):
    model = ... # Define your achitecure
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss=..., # Choose your loss
        metrics=[METRIC_NAME],
    )

    model.fit(
        x_train, y_train, epochs=1
    )  # Run with 1 epoch to speed things up for demo purposes
    _, accuracy = model.evaluate(..., ...) # to complete
    return accuracy

Pour chaque run, logger les hyper-paramètres et la métrique choisie.

In [ ]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_NAME, accuracy, step=1)

## Start runs and log them all under one parent directory

Lancer l'hyper-paramétrage en réalisant plusieurs run avec un set d'hyper-paramètres différents.

Utiliser un grid search avec peu de combinaisons pour accélérer le processus. Pour les range de valeurs continues utiliser seulement le min et le max.

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
            }
            run_name = "run-%d" % session_num
            print("--- Starting trial: %s" % run_name)
            print({h.name: hparams[h] for h in hparams})
            run("logs/hparam_tuning/" + run_name, hparams)
            session_num += 1

## Create Vertex AI TensorBoard
Créer une instance Tensorboard pour visualiser votre entrainement.

Plus de détail ici : [Create a Vertex AI TensorBoard instance](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-setup#create-tensorboard-instance).

In [ ]:
# Set the display name for your tensorboard instance

TENSORBOARD_NAME = f"supaero-vertex-course-1-tb-{PROJECT_ID}-{your_name}"  # @param {type:"string"}

tensorboard = aiplatform.Tensorboard.create(
    display_name=TENSORBOARD_NAME, project=PROJECT_ID, location=LOCATION
)
TENSORBOARD_RESOURCE_NAME = tensorboard.gca_resource.name
print("TensorBoard resource name:", TENSORBOARD_RESOURCE_NAME)

In [ ]:
EXPERIMENT_NAME = f"supaero-vertex-course-1-5-{PROJECT_ID}-{your_name}"  # @param {type:"string"}

Upload the log to your Vertex AI TensorBoard.

In [ ]:
!tb-gcp-uploader --one_shot=True --tensorboard_resource_name=$TENSORBOARD_RESOURCE_NAME --logdir="logs/hparam_tuning/" --experiment_name=$EXPERIMENT_NAME

## Visualiser vos Résultats

Cliquer sur le lien générer par la cellule ci-dessus ou aller dans le portail Vertex AI. Les fonctionnalités disponibles sont :

- Filtre et tri sur les hyper-paramètres, les métriques, et le status des runs
- Sélection des runs

Le dashboard HParams dashboard possède 3 vues différentes :

* Listing des runs, hyper-paramètres et métriques :  **Table View** 
* Comparaison de runs interactives via les hyper-paramètres :  **Parallel Coordinates View** 
* Comparaison de runs interactives via les métriques : **Scatter Plot Matrix View** shows plots 



## Your Turn

Quels hyper-paramètres permettent d'obtenir les meilleurs performances ? (check via le tensorboard)

Est-ce logique ?

Your answer : 